# INSTALL LIBRARIES

In [1]:
!pip install mlflow==2.13.2 sagemaker-mlflow==0.1.0

# LOAD LIBRARIES

In [9]:
import mlflow
import mlflow.xgboost
import mlflow.data
from mlflow.data.pandas_dataset import PandasDataset

import xgboost as xgb
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd
import os

# SETTINGS

In [3]:
mlflow_arn = "arn:aws:sagemaker:eu-west-1:211125740051:mlflow-tracking-server/Sample-server"
mlflow_experiment_name = "02-sample-experiment"

# SET MLFLOW

In [4]:
mlflow.set_tracking_uri(mlflow_arn)
mlflow.set_experiment(mlflow_experiment_name)

<Experiment: artifact_location='s3://sagemaker-bucket-ds/ML_FLOW/35', creation_time=1719996896821, experiment_id='35', last_update_time=1719996896821, lifecycle_stage='active', name='02-sample-experiment', tags={}>

# LOAD DATA

In [22]:
# Load Diabetes dataset
data = load_diabetes()
X = data.data
y = data.target

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

df_train = pd.DataFrame(data=X_train, columns=data.feature_names)
df_train['target'] = y_train
dataset_train = mlflow.data.from_pandas(df_train, source="load_diabetes")


df_test = pd.DataFrame(data=X_test, columns=data.feature_names)
df_test['target'] = y_test
dataset_test = mlflow.data.from_pandas(df_test, source="load_diabetes")

/opt/conda/lib/python3.10/site-packages/mlflow/data/dataset_source_registry.py:150: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'load_diabetes'. Exception: 
  return _dataset_source_registry.resolve(
/opt/conda/lib/python3.10/site-packages/mlflow/data/dataset_source_registry.py:150: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/opt/conda/lib/python3.10/site-packages/mlflow/data/dataset_source_registry.py:150: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'load_diabetes'. Exception: 
  return _dataset_source_registry.resolve(
/opt/conda/lib/python3.10/site-packages/mlflow/data/dataset_source_registry.py:150: UserWarning: The specified dataset source can be interpreted in multiple ways

# BUILD A MODEL

In [24]:
# Create and train model
model = xgb.XGBRegressor(n_estimators=100, max_depth=1, random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=1, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [25]:
# Start a new MLflow run
with mlflow.start_run(run_name = "run-with-data") as run:
    run_id = run.info.run_id
    print(f"Run ID: {run_id}")

    # Log parameters directly from the model
    params = model.get_params()
    for param, value in params.items():
        mlflow.log_param(param, value)

    mlflow.log_input(dataset_train, context="training")
    mlflow.log_input(dataset_test, context="test")
    
    # Log the model
    mlflow.xgboost.log_model(model, "model")

print("Logging completed.")

Run ID: 651f69fe227f4cedb5d9e9e51e3efe5e
Logging completed.
